In [80]:
import pandas as pd
import numpy as np

from scipy import stats
from  datetime import datetime, timedelta, date

!pip install yfinance
import yfinance as yf

In [107]:
rates = pd.read_html("https://www.global-rates.com/en/interest-rates/libor/libor.aspx")[9][1][2:].apply(lambda x : 0.01*float(x[:-2])).values
# LIBOR Rate for 1, 3, 6, 12 months
print(rates)
vol = yf.Ticker("^VIX").history().iloc[-1]['Close']*0.01
# Volatility from VIX Index

cur_date = date.today()

[0.00162   0.0046386 0.0075829 0.0125771]


In [141]:
symbols = ["AAPL"]

data = []


for symbol in symbols:

    ticker = yf.Ticker(symbol)
    S = ticker.history(auto_adjust=False).iloc[-1]['Close']
    q = (4 * ticker.dividends[-1])/S

    for expiry_date in ticker.options:
        
        T = np.busday_count(cur_date, expiry_date) / 250

        if T <= 1/12:
            rf = rates[0]
        elif T <= 3/12:
            rf = rates[1]
        elif T <= 6/12:
            rf = rates[2]
        else:
            rf = rates[3]


        df = pd.DataFrame(ticker.option_chain(expiry_date).calls).sort_values(by='lastTradeDate', ascending=False).groupby("strike").min().reset_index()
        

        for _ in range(len(df)):
            K, P, actual_imp_vol = df.iloc[_][['strike', 'lastPrice', 'impliedVolatility']]        

            data.append([S, K, T, rf, vol, actual_imp_vol, P])    

In [142]:
data = np.array(data)
final_df = pd.DataFrame(data=data, columns=["Stock", "Strike", "Maturity", "Int Rate", "Vol", "Imp Vol", "Option Price"])

In [143]:
final_df

,Stock,Strike,Maturity,Int Rate,Vol,Imp Vol,Option Price
0,164.320007,105.0,0.012,0.001620,0.2881,2.179692,59.15
1,164.320007,110.0,0.012,0.001620,0.2881,2.669925,64.50
2,164.320007,115.0,0.012,0.001620,0.2881,1.640627,49.35
3,164.320007,120.0,0.012,0.001620,0.2881,2.353520,47.52
4,164.320007,125.0,0.012,0.001620,0.2881,1.406253,39.43
...,...,...,...,...,...,...,...
880,164.320007,260.0,1.992,0.012577,0.2881,0.302284,5.55
881,164.320007,270.0,1.992,0.012577,0.2881,0.301948,4.60
882,164.320007,280.0,1.992,0.012577,0.2881,0.301856,3.75
883,164.320007,290.0,1.992,0.012577,0.2881,0.301460,3.25
